# Planet monthly mosaic data

* **Products used:** 
Planet API


## Background

## Description
This notebook demonstrates how to how to use the Planet monthly mosaic data:

1. 

***

**Important details:**

* Datacube product names: `Planet Monthly Mosaic`
* Date-range: 2020-09 &ndash; present
* Spatial resolution: 


## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load packages
Import Python packages that are used for the analysis.

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
#pip install localtileserver

In [3]:
from planet_basemap import loadplanet

In [4]:
loadplanet()

Map(center=[6.7424885, -1.6934010000000002], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_…